In [0]:
# Testing GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12246223215479498568
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6627290954954079748
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5202868545103632765
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11330115994
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2108973114249173841
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [0]:
import pandas as pd
import numpy as np
from numpy import dstack
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
s1 = pd.read_pickle('pred_probability_xCEPTION.pkl')
s2 = pd.read_pickle('pred_probability_inception.pkl')
s3 = pd.read_pickle('pred_probability_resnet.pkl')
s4 = pd.read_pickle('pred_probability_vgg19.pkl')
s = [s2, s3, s4]

for x in s:
  x = x.astype('float32')
  print(x.shape)
st = s1
st = dstack((st,s2))
st = dstack((st,s3))
st = dstack((st,s4))
#st = dstack((st,s4))
print(st.shape)

t1 = pd.read_pickle('train_probability_xCEPTION.pkl')
t2 = pd.read_pickle('train_probability_inception.pkl')
t3 = pd.read_pickle('train_probability_resnet.pkl')
t4 = pd.read_pickle('train_probability_vgg19.pkl')

t = [t2, t3, t4]
tr = t1
tr = dstack((tr,t2))
tr = dstack((tr,t3))
tr = dstack((tr,t4))
print(tr.shape)

(10000, 10)
(10000, 10)
(10000, 10)
(10000, 10, 4)
(50000, 10, 4)


In [0]:
st = st.reshape((st.shape[0], st.shape[1]*st.shape[2]))
print(st.shape)

tr = tr.reshape((tr.shape[0], tr.shape[1]*tr.shape[2]))
print(tr.shape)
 

(10000, 40)
(50000, 40)


In [0]:
from xgboost import XGBClassifier
model = XGBClassifier()
df = pd.read_csv('train_max_y.csv')
labels = df.iloc[:,1]
model.fit(tr,labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
y_pred = model.predict(st)
print(y_pred)

[9 6 4 ... 7 6 8]


In [0]:
pred_class = np.argmax(y_pred, axis=1)
test_preds = pd.DataFrame()
test_preds['Id'] = range(10000)
test_preds['Label'] = pred_class
print(test_preds.head(20))

    Id  Label
0    0      9
1    1      6
2    2      4
3    3      9
4    4      5
5    5      8
6    6      2
7    7      4
8    8      6
9    9      8
10  10      4
11  11      9
12  12      8
13  13      9
14  14      9
15  15      6
16  16      8
17  17      7
18  18      4
19  19      6


In [0]:
test_preds.to_csv("test-s3.csv", index=False)
from google.colab import files

files.download("test-s3.csv")


In [0]:
import lightgbm as lgb
params = {
 'task': 'train'
 , 'boosting_type': 'gbdt'
 , 'objective': 'multiclass'
 , 'num_class': 10
 , 'metric': 'multi_logloss'
 , 'min_data': 1
 , 'verbose': -1
}

lgb_train = lgb.Dataset(tr[:40000,:], labels[:40000])
lgb_eval = lgb.Dataset(tr[40000:,:], labels[40000:], reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

y_pred = gbm.predict(tr[:40000], num_iteration=gbm.best_iteration)
print('train_acc: ', np.mean(np.argmax(y_pred, axis=1)==labels[:40000]))
y_pred = gbm.predict(tr[40000:], num_iteration=gbm.best_iteration)
print('test_acc: ', np.mean(np.argmax(y_pred, axis=1)==labels[40000:]))

y_pred = gbm.predict(st, num_iteration=gbm.best_iteration)

[1]	valid_0's multi_logloss: 1.45581
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.24334
[3]	valid_0's multi_logloss: 1.07982
[4]	valid_0's multi_logloss: 0.94714
[5]	valid_0's multi_logloss: 0.836277
[6]	valid_0's multi_logloss: 0.741918
[7]	valid_0's multi_logloss: 0.660557
[8]	valid_0's multi_logloss: 0.589742
[9]	valid_0's multi_logloss: 0.527677
[10]	valid_0's multi_logloss: 0.472984
[11]	valid_0's multi_logloss: 0.424583
[12]	valid_0's multi_logloss: 0.381603
[13]	valid_0's multi_logloss: 0.343334
[14]	valid_0's multi_logloss: 0.309181
[15]	valid_0's multi_logloss: 0.278643
[16]	valid_0's multi_logloss: 0.251298
[17]	valid_0's multi_logloss: 0.226777
[18]	valid_0's multi_logloss: 0.204762
[19]	valid_0's multi_logloss: 0.184979
[20]	valid_0's multi_logloss: 0.167188
[21]	valid_0's multi_logloss: 0.151178
[22]	valid_0's multi_logloss: 0.136761
[23]	valid_0's multi_logloss: 0.123772
[24]	valid_0's multi_logloss: 0.112065
[25]	valid_0's m